# Performance graphs in the driving simulator

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle


def load_obj(name,dir_name):
    with open(dir_name + 'obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

    
plt.rcParams.update({'font.size': 12})
plt.rcParams.update({'font.size': 12})

In [ ]:
plt.rcParams.update({'font.size': 12})


##  Function definitions

In [ ]:
def make_graphs_drv():
    
    repeats = 10
    sample_step = 10
    iters1 = 0
    iters2 = 0
    dlistE = []
    dlistBB1 = []
    
    for i in range(repeats):
        try:
            dlistE.append(load_obj("ellips_values"+str(i),'Autonomous driving simulation/'))
            if iters1 == 0:
                iters1 = len(dlistE[0][0,"expert_values"])
        except:
            pass
    dlistBB1 = []
    for i in range(repeats):
        try:
            dlistBB1.append(load_obj("es_values"+str(i),'Autonomous driving simulation/'))
            if iters2 == 0:
                iters2 = len(dlistBB1[0][0,"expert_values"])
        except:
            pass
        
    # Build data Ellipsoid:
    expert_valuesE =    np.zeros((repeats,iters1))
    agent_valuesE =     np.zeros((repeats,iters1))
    cumm_regretE =      np.zeros((repeats,iters1))
    contexts_seenE =    np.zeros((repeats,iters1))
    contexts_seen_minE = np.inf
    for trainset in range(repeats):
        for d in dlistE:
            if (trainset,"expert_values") in d:
                expert_valuesE[trainset] =    d[trainset,"expert_values"]
                agent_valuesE[trainset] =     d[trainset,"agent_values"]
                cumm_regretE[trainset] =      d[trainset,"cum_regret"]
                contexts_seenE[trainset] =    d[trainset,"contexts_seen"]
                contexts_seen_minE = min(contexts_seen_minE, contexts_seenE[trainset][-1])

    contexts_seen_minE -= contexts_seen_minE%sample_step
    contexts_seen_minE = int(contexts_seen_minE)+sample_step
    test_expert_valueE = np.zeros((repeats,int(contexts_seen_minE/sample_step)))
    test_agent_valueE = np.zeros((repeats,int(contexts_seen_minE/sample_step)))
    TESTS = False
    
    for trainset in range(repeats):
        for d in dlistE:
                if (trainset,"test_value",0) in d:
                    TESTS = True
                    for contexts in range(0,contexts_seen_minE,sample_step):
                        test_expert_valueE[trainset][int(contexts/sample_step)] = d["test_value"]
                        test_agent_valueE[trainset][int(contexts/sample_step)] = d[trainset,"test_value",contexts]

    expert_valuesE = expert_valuesE[~np.all(expert_valuesE == 0, axis=1)]
    agent_valuesE = agent_valuesE[~np.all(agent_valuesE == 0, axis=1)]
    cumm_regretE = cumm_regretE[~np.all(cumm_regretE == 0, axis=1)]
    contexts_seenE = contexts_seenE[~np.all(contexts_seenE == 0, axis=1)]
    test_expert_valueE = test_expert_valueE[~np.all(test_expert_valueE == 0, axis=1)]
    test_agent_valueE = test_agent_valueE[~np.all(test_agent_valueE == 0, axis=1)] 
    
    expert_values_stdE =     expert_valuesE.std(axis=0)
    agent_values_stdE =      agent_valuesE.std(axis=0)
    expert_values_meanE =    expert_valuesE.mean(axis=0)
    agent_values_meanE =     agent_valuesE.mean(axis=0)
    cumm_regret_meanE =      cumm_regretE.mean(axis=0)
    cumm_regret_stdE =       cumm_regretE.std(axis=0)
    contexts_seen_stdE =     contexts_seenE.std(axis=0)
    contexts_seen_meanE =    contexts_seenE.mean(axis=0)
    

    # Build data Blackbox 1st loss:
    expert_valuesBB1 =    np.zeros((repeats,iters2))
    agent_valuesBB1 =     np.zeros((repeats,iters2))
    cumm_regretBB1 =      np.zeros((repeats,iters2))
    contexts_seenBB1 =    np.zeros((repeats,iters2))
    contexts_seen_minBB1 = np.inf
    adv = 0
    for trainset in range(repeats):
        for d in dlistBB1:
            if (trainset,"expert_values") in d:
                expert_valuesBB1[adv] =    d[trainset,"expert_values"]
                agent_valuesBB1[adv] =     d[trainset,"agent_values"]
                cumm_regretBB1[adv] =      d[trainset,"cum_regret"]
                contexts_seenBB1[adv] =    d[trainset,"contexts_seen"]
                contexts_seen_minBB1 = min(contexts_seen_minBB1, contexts_seenBB1[adv][-1])
                adv += 1
                
    contexts_seen_minBB1 -= contexts_seen_minBB1%sample_step
    contexts_seen_minBB1 = int(contexts_seen_minBB1)+sample_step
    test_expert_valueBB1 = np.zeros((repeats,int(contexts_seen_minBB1/sample_step)))
    test_agent_valueBB1 = np.zeros((repeats,int(contexts_seen_minBB1/sample_step)))
    
    TESTS = False
    adv = 0
    for trainset in range(repeats):
        for d in dlistBB1:
                if (trainset,"test_value",0) in d:
                    TESTS = True
                    for contexts in range(0,contexts_seen_minBB1,sample_step):
                        test_expert_valueBB1[adv][int(contexts/sample_step)] = d["test_value"]
                        test_agent_valueBB1[adv][int(contexts/sample_step)] = d[trainset,"test_value",contexts]
                    adv += 1

    expert_valuesBB1 = expert_valuesBB1[~np.all(expert_valuesBB1 == 0, axis=1)]
    agent_valuesBB1 = agent_valuesBB1[~np.all(agent_valuesBB1 == 0, axis=1)]
    cumm_regretBB1 = cumm_regretBB1[~np.all(cumm_regretBB1 == 0, axis=1)]
    contexts_seenBB1 = contexts_seenBB1[~np.all(contexts_seenBB1 == 0, axis=1)]
    test_expert_valueBB1 = test_expert_valueBB1[~np.all(test_expert_valueBB1 == 0, axis=1)]
    test_agent_valueBB1 = test_agent_valueBB1[~np.all(test_agent_valueBB1 == 0, axis=1)]    
    
    expert_values_stdBB1 =     expert_valuesBB1.std(axis=0)
    agent_values_stdBB1 =      agent_valuesBB1.std(axis=0)
    expert_values_meanBB1 =    expert_valuesBB1.mean(axis=0)
    agent_values_meanBB1 =     agent_valuesBB1.mean(axis=0)
    cumm_regret_meanBB1 =      cumm_regretBB1.mean(axis=0)
    cumm_regret_stdBB1 =       cumm_regretBB1.std(axis=0)
    contexts_seen_stdBB1 =     contexts_seenBB1.std(axis=0)
    contexts_seen_meanBB1 =    contexts_seenBB1.mean(axis=0)
    
    if TESTS:
        # Value graph (contexts)
        fig = plt.figure(figsize=(4,3))
        p = fig.add_subplot(111)
        
        test_expert_value_meanE = test_expert_valueE.mean(axis=0)
        test_expert_value_stdE = test_expert_valueE.std(axis=0)
        test_agent_value_meanE = test_agent_valueE.mean(axis=0)
        test_agent_value_stdE = test_agent_valueE.std(axis=0)

        p.plot(list(range(0,contexts_seen_minE,sample_step)), test_expert_value_meanE, label="Expert", color='black')
        p.plot(list(range(0,contexts_seen_minE,sample_step)), test_agent_value_meanE, label="Ellipsoid", color='g')

        p.fill_between(list(range(0,contexts_seen_minE,sample_step)), test_expert_value_meanE - test_expert_value_stdE, test_expert_value_meanE + test_expert_value_stdE, color='black', alpha=0.1)
        p.fill_between(list(range(0,contexts_seen_minE,sample_step)), test_agent_value_meanE - test_agent_value_stdE, test_agent_value_meanE + test_agent_value_stdE, color='g', alpha=0.1)

        test_expert_value_meanBB1 = test_expert_valueBB1.mean(axis=0)
        test_expert_value_stdBB1 = test_expert_valueBB1.std(axis=0)
        test_agent_value_meanBB1 = test_agent_valueBB1.mean(axis=0)
        test_agent_value_stdBB1 = test_agent_valueBB1.std(axis=0)

        p.plot(list(range(0,contexts_seen_minBB1,sample_step)), test_agent_value_meanBB1, label="ES", color='r')

        p.fill_between(list(range(0,contexts_seen_minBB1,sample_step)), test_agent_value_meanBB1 - test_agent_value_stdBB1, test_agent_value_meanBB1 + test_agent_value_stdBB1, color='r', alpha=0.1)    
    
        p.set_xlabel("Demonstrations")
        p.set_ylabel("Value")
        p.tick_params('y', colors='k')
#         p.set_ylim(0.014,0.021)
        p.legend(loc=8)
        plt.show()
        
    
# Regret (time)
    fig = plt.figure(figsize=(4,3))
    p = fig.add_subplot(111)
    
    p.plot(list(range(iters1)), cumm_regret_meanE, label="Ellipsoid", color='g')
    p.fill_between(list(range(iters1)), cumm_regret_meanE - cumm_regret_stdE, cumm_regret_meanE + cumm_regret_stdE, color='g', alpha=0.07)
    p.plot(list(range(iters2)), cumm_regret_meanBB1, label="ES", color='r')
    p.fill_between(list(range(iters2)), cumm_regret_meanBB1 - cumm_regret_stdBB1, cumm_regret_meanBB1 + cumm_regret_stdBB1, color='r', alpha=0.07)
    
    
    p.set_xlabel("Time")
    p.set_ylabel("Cumulative Regret")
#     p.legend(loc=4)
#     plt.title("Cumulative regret over time")
    plt.show()


    fig = plt.figure(figsize=(4,3))
    p2 = fig.add_subplot(111)

    p2.plot(list(range(iters1)), contexts_seen_meanE, label="Ellipsoid", color='g')
    p2.fill_between(list(range(iters1)), contexts_seen_meanE - contexts_seen_stdE, contexts_seen_meanE + contexts_seen_stdE, color='g', alpha=0.1)

    
    p2.plot(list(range(iters2)), contexts_seen_meanBB1, label="ES", color='r')
    p2.fill_between(list(range(iters2)), contexts_seen_meanBB1 - contexts_seen_stdBB1, contexts_seen_meanBB1 + contexts_seen_stdBB1, color='r', alpha=0.1)

    p2.set_ylabel("Demonstrations")
    p2.tick_params('y', colors='g')
    p2.set_xlabel("Time")

#     plt.title("Demonstrations required over time")
#     p2.legend(loc=2)

    plt.show()


In [ ]:
def make_graphs_drv_non_linear():
    
    repeats = 10
    sample_step = 5
    iters1 = 0
    iters2 = 0
    dlistE = []
    for i in range(repeats):
        try:
            dlistE.append(load_obj("ellips_nl_values"+str(i),'Autonomous driving simulation/'))
            if iters1 == 0:
                iters1 = len(dlistE[0][0,"expert_values"])
        except:
            pass
    dlistBB2 = []
    for i in range(repeats):
        try:
            dlistBB2.append(load_obj("es_nl_values"+str(i),'Autonomous driving simulation/'))
            if iters2 == 0:
                iters2 = len(dlistBB2[0][0,"expert_values"])
        except:
            pass
    
    # Build data ellipsoid
    expert_valuesE =    np.zeros((repeats,iters1))
    agent_valuesE =     np.zeros((repeats,iters1))
    cum_regretE =      np.zeros((repeats,iters1))
    contexts_seenE =    np.zeros((repeats,iters1))
    contexts_seen_minE = np.inf
    for trainset in range(repeats):
        for d in dlistE:
            if (trainset,"expert_values") in d:
                expert_valuesE[trainset] =    d[trainset,"expert_values"]
                agent_valuesE[trainset] =     d[trainset,"agent_values"]
                cum_regretE[trainset] =       d[trainset,"cum_regret"]
                contexts_seenE[trainset] =    d[trainset,"contexts_seen"]
                contexts_seen_minE = min(contexts_seen_minE, contexts_seenE[trainset][-1])

    contexts_seen_minE = int(contexts_seen_minE)
    test_expert_valueE= np.zeros((repeats,contexts_seen_minE+1))
    test_agent_valueE = np.zeros((repeats,contexts_seen_minE+1))

    TESTS = False
    for trainset in range(repeats):
        for d in dlistE:
                if (trainset,"test_value",sample_step) in d:
                    TESTS = True
                    for contexts in range(0,contexts_seen_minE+1):
                        test_expert_valueE[trainset][contexts] = d["test_value"]
                        test_agent_valueE[trainset][contexts] = d[trainset,"test_value",contexts]

    expert_valuesE = expert_valuesE[~np.all(expert_valuesE == 0, axis=1)]
    agent_valuesE = agent_valuesE[~np.all(agent_valuesE == 0, axis=1)]
    cum_regretE = cum_regretE[~np.all(cum_regretE == 0, axis=1)]
    contexts_seenE = contexts_seenE[~np.all(contexts_seenE == 0, axis=1)]
    test_expert_valueE = test_expert_valueE[~np.all(test_expert_valueE == 0, axis=1)]
    test_agent_valueE = test_agent_valueE[~np.all(test_agent_valueE == 0, axis=1)]

    expert_values_stdE =     expert_valuesE.std(axis=0)
    agent_values_stdE =      agent_valuesE.std(axis=0)
    expert_values_meanE =    expert_valuesE.mean(axis=0)
    agent_values_meanE =     agent_valuesE.mean(axis=0)
    cum_regret_meanE =       cum_regretE.mean(axis=0)
    cum_regret_stdE =        cum_regretE.std(axis=0)
    contexts_seen_stdE =     contexts_seenE.std(axis=0)
    contexts_seen_meanE =    contexts_seenE.mean(axis=0)

    # Build data Blackbox 2nd loss
    expert_valuesBB2 =    np.zeros((repeats,iters2))
    agent_valuesBB2 =     np.zeros((repeats,iters2))
    cum_regretBB2 =       np.zeros((repeats,iters2))
    contexts_seenBB2 =    np.zeros((repeats,iters2))
    contexts_seen_minBB2 = np.inf
    for trainset in range(repeats):
        for d in dlistBB2:
            if (trainset,"expert_values") in d and trainset != 14:
                expert_valuesBB2[trainset] =    d[trainset,"expert_values"]
                agent_valuesBB2[trainset] =     d[trainset,"agent_values"]
                cum_regretBB2[trainset] =       d[trainset,"cum_regret"]
                contexts_seenBB2[trainset] =    d[trainset,"contexts_seen"]
                contexts_seen_minBB2 = min(contexts_seen_minBB2, contexts_seenBB2[trainset][-1])

    contexts_seen_minBB2 = int(contexts_seen_minBB2)
    test_expert_valueBB2 = np.zeros((repeats,contexts_seen_minBB2+1))
    test_agent_valueBB2 = np.zeros((repeats,contexts_seen_minBB2+1))

    TBSTS = False
    for trainset in range(repeats):
        for d in dlistBB2:
                if (trainset,"test_value",sample_step) in d and trainset != 14:
                    TBSTS = True
                    for contexts in range(0,contexts_seen_minBB2+1):
                        test_expert_valueBB2[trainset][contexts] = d["test_value"]
                        test_agent_valueBB2[trainset][contexts] = d[trainset,"test_value",contexts]

    expert_valuesBB2 = expert_valuesBB2[~np.all(expert_valuesBB2 == 0, axis=1)]
    agent_valuesBB2 = agent_valuesBB2[~np.all(agent_valuesBB2 == 0, axis=1)]
    cum_regretBB2 = cum_regretBB2[~np.all(cum_regretBB2 == 0, axis=1)]
    contexts_seenBB2 = contexts_seenBB2[~np.all(contexts_seenBB2 == 0, axis=1)]
    test_expert_valueBB2 = test_expert_valueBB2[~np.all(test_expert_valueBB2 == 0, axis=1)]
    test_agent_valueBB2 = test_agent_valueBB2[~np.all(test_agent_valueBB2 == 0, axis=1)]

    expert_values_stdBB2 =     expert_valuesBB2.std(axis=0)
    agent_values_stdBB2 =      agent_valuesBB2.std(axis=0)
    expert_values_meanBB2 =    expert_valuesBB2.mean(axis=0)
    agent_values_meanBB2 =     agent_valuesBB2.mean(axis=0)
    cum_regret_meanBB2 =       cum_regretBB2.mean(axis=0)
    cum_regret_stdBB2 =        cum_regretBB2.std(axis=0)
    contexts_seen_stdBB2 =     contexts_seenBB2.std(axis=0)
    contexts_seen_meanBB2 =    contexts_seenBB2.mean(axis=0)

    if TESTS:
        # Value graph (contexts)
        fig = plt.figure(figsize=(4,3))
        p = fig.add_subplot(111)
        
        test_expert_value_meanE = test_expert_valueE.mean(axis=0)
        test_expert_value_stdE = test_expert_valueE.std(axis=0)
        test_agent_value_meanE = test_agent_valueE.mean(axis=0)
        test_agent_value_stdE = test_agent_valueE.std(axis=0)

        p.plot(list(range(contexts_seen_minE+1)), test_expert_value_meanE, label="Expert", color='k')
        p.plot(list(range(contexts_seen_minE+1)), test_agent_value_meanE, label="Ellipsoid", color='g')

        p.fill_between(list(range(contexts_seen_minE+1)), test_expert_value_meanE - test_expert_value_stdE, test_expert_value_meanE + test_expert_value_stdE, color='g', alpha=0.1)
        p.fill_between(list(range(contexts_seen_minE+1)), test_agent_value_meanE - test_agent_value_stdE, test_agent_value_meanE + test_agent_value_stdE, color='g', alpha=0.1)

        test_expert_value_meanBB2 = test_expert_valueBB2.mean(axis=0)
        test_expert_value_stdBB2 = test_expert_valueBB2.std(axis=0)
        test_agent_value_meanBB2 = test_agent_valueBB2.mean(axis=0)
        test_agent_value_stdBB2 = test_agent_valueBB2.std(axis=0)

#         p.plot(list(range(0,contexts_seen_minBB2,5)), test_expert_value_meanBB2, label="Expert", color='b')
        p.plot(list(range(contexts_seen_minBB2+1)), test_agent_value_meanBB2, label="ES", color='b')

#         p.fill_between(list(range(0,contexts_seen_minBB2,5)), test_expert_value_meanBB2 - test_expert_value_stdBB2, test_expert_value_meanBB2 + test_expert_value_stdBB2, color='b', alpha=0.1)
        p.fill_between(list(range(contexts_seen_minBB2+1)), test_agent_value_meanBB2 - test_agent_value_stdBB2, test_agent_value_meanBB2 + test_agent_value_stdBB2, color='b', alpha=0.1)

        p.set_xlabel("Demonstrations")
        p.set_ylabel("Value")
        p.tick_params('y', colors='k')

#         plt.title("Avg. value on test set")
        plt.xlim(0, 95)
        p.legend(loc=1)
        plt.show()
    
    # demonstrations (time)
    fig = plt.figure(figsize=(4,3))
    p2 = fig.add_subplot(111)

    p2.plot(list(range(iters1)), contexts_seen_meanE, label="Ellipsoid", color='g')
    p2.fill_between(list(range(iters1)), contexts_seen_meanE - contexts_seen_stdE, contexts_seen_meanE + contexts_seen_stdE, color='g', alpha=0.1)
    p2.set_ylabel("Contexts seen")
    p2.tick_params('y', colors='g')
    
    p2.plot(list(range(iters2)), contexts_seen_meanBB2, label="ES", color='b')
    p2.fill_between(list(range(iters2)), contexts_seen_meanBB2 - contexts_seen_stdBB2, contexts_seen_meanBB2 + contexts_seen_stdBB2, color='b', alpha=0.1)
    p2.set_ylabel("Demonstrations")
    p2.tick_params('y', colors='k')

    p2.set_xlabel("Time")
    p2.legend(loc=2)
    plt.show()
    
    
    # Regret (time)
    fig = plt.figure(figsize=(4,3))
    p = fig.add_subplot(111)

    p.plot(list(range(iters1)), cum_regret_meanE, label="Ellipsoid", color='g')
    p.fill_between(list(range(iters1)), cum_regret_meanE - cum_regret_stdE, cum_regret_meanE + cum_regret_stdE, color='g', alpha=0.1)

    p.plot(list(range(iters2)), cum_regret_meanBB2, label="ES", color='b')
    p.fill_between(list(range(iters2)), cum_regret_meanBB2 - cum_regret_stdBB2, cum_regret_meanBB2 + cum_regret_stdBB2, color='b', alpha=0.1)

    p.set_xlabel("Time")
    p.set_ylabel("Regret")
    p.legend(loc=2)
    plt.show()


In [ ]:
def make_graphs_med():
    
    repeats = 5
    sample_step = 10
    iters1 = 0
    iters2 = 0
    
    dlistE = []
    dlistBB1 = []
    
    accurE = []
    accurBB1 = []

    # plot accuracy:
    for i in range(repeats):
        try:
            accurE.append(np.load("./Dynamic treatment regime/obj/accuracy_ellips" + str(i) + ".npy"))
            
        except:
            pass
        
    for i in range(repeats):
        try:
            accurBB1.append(np.load("./Linear/Dynamic treatment regime/obj/accuracy_es" + str(i) + ".npy"))
            
        except:
            pass       
        
    for i in range(repeats):
        try:
            dlistE.append(load_obj("ellips_values"+str(i),'Dynamic treatment regime/'))
            if iters1 == 0:
                iters1 = len(dlistE[0][0,"expert_values"])
        except:
            pass
    dlistBB1 = []
    for i in range(repeats):
        try:
            dlistBB1.append(load_obj("es_values"+str(i),'Dynamic treatment regime/'))
            if iters2 == 0:
                iters2 = len(dlistBB1[0][0,"expert_values"])
        except:
            pass

    # Build data Ellipsoid:
    expert_valuesE =    np.zeros((repeats,iters1))
    agent_valuesE =     np.zeros((repeats,iters1))
    cumm_regretE =      np.zeros((repeats,iters1))
    contexts_seenE =    np.zeros((repeats,iters1))
    contexts_seen_minE = np.inf
    for trainset in range(repeats):
        for d in dlistE:
            if (trainset,"expert_values") in d:
                expert_valuesE[trainset] =    d[trainset,"expert_values"]
                agent_valuesE[trainset] =     d[trainset,"agent_values"]
                cumm_regretE[trainset] =      d[trainset,"cum_regret"]
                contexts_seenE[trainset] =    d[trainset,"contexts_seen"]
                contexts_seen_minE = min(contexts_seen_minE, contexts_seenE[trainset][-1])

    contexts_seen_minE -= contexts_seen_minE%sample_step
    contexts_seen_minE = int(contexts_seen_minE)+sample_step
    test_expert_valueE = np.zeros((repeats,int(contexts_seen_minE/sample_step)))
    test_agent_valueE = np.zeros((repeats,int(contexts_seen_minE/sample_step)))
    TESTS = False
    
    
    for trainset in range(repeats):
        for d in dlistE:
                if (trainset,"test_value",0) in d:
                    TESTS = True
                    for contexts in range(0,contexts_seen_minE,sample_step):
                        test_expert_valueE[trainset][int(contexts/sample_step)] = d["test_value"]
                        test_agent_valueE[trainset][int(contexts/sample_step)] = d[trainset,"test_value",contexts]

    expert_valuesE = expert_valuesE[~np.all(expert_valuesE == 0, axis=1)]
    agent_valuesE = agent_valuesE[~np.all(agent_valuesE == 0, axis=1)]
    cumm_regretE = cumm_regretE[~np.all(cumm_regretE == 0, axis=1)]
    contexts_seenE = contexts_seenE[~np.all(contexts_seenE == 0, axis=1)]
    test_expert_valueE = test_expert_valueE[~np.all(test_expert_valueE == 0, axis=1)]
    test_agent_valueE = test_agent_valueE[~np.all(test_agent_valueE == 0, axis=1)
    
    expert_values_stdE =     expert_valuesE.std(axis=0)
    agent_values_stdE =      agent_valuesE.std(axis=0)
    expert_values_meanE =    expert_valuesE.mean(axis=0)
    agent_values_meanE =     agent_valuesE.mean(axis=0)
    cumm_regret_meanE =      cumm_regretE.mean(axis=0)
    cumm_regret_stdE =       cumm_regretE.std(axis=0)
    contexts_seen_stdE =     contexts_seenE.std(axis=0)
    contexts_seen_meanE =    contexts_seenE.mean(axis=0)
    

    # Build data Blackbox 1st loss:
    expert_valuesBB1 =    np.zeros((repeats,iters2))
    agent_valuesBB1 =     np.zeros((repeats,iters2))
    cumm_regretBB1 =      np.zeros((repeats,iters2))
    contexts_seenBB1 =    np.zeros((repeats,iters2))
    contexts_seen_minBB1 = np.inf
    adv = 0
    for trainset in range(repeats):
        for d in dlistBB1:
            if (trainset,"expert_values") in d:
                expert_valuesBB1[adv] =    d[trainset,"expert_values"]
                agent_valuesBB1[adv] =     d[trainset,"agent_values"]
                cumm_regretBB1[adv] =      d[trainset,"cum_regret"]
                contexts_seenBB1[adv] =    d[trainset,"contexts_seen"]
                contexts_seen_minBB1 = min(contexts_seen_minBB1, contexts_seenBB1[adv][-1])
                adv += 1
                
    contexts_seen_minBB1 -= contexts_seen_minBB1%sample_step
    contexts_seen_minBB1 = int(contexts_seen_minBB1)+sample_step
    test_expert_valueBB1 = np.zeros((repeats,int(contexts_seen_minBB1/sample_step)))
    test_agent_valueBB1 = np.zeros((repeats,int(contexts_seen_minBB1/sample_step)))
    
    TESTS = False
    adv = 0
    for trainset in range(repeats):
        for d in dlistBB1:
                if (trainset,"test_value",0) in d:
                    TESTS = True
                    for contexts in range(0,contexts_seen_minBB1,sample_step):
                        test_expert_valueBB1[adv][int(contexts/sample_step)] = d["test_value"]
                        test_agent_valueBB1[adv][int(contexts/sample_step)] = d[trainset,"test_value",contexts]
                    adv += 1

    expert_valuesBB1 = expert_valuesBB1[~np.all(expert_valuesBB1 == 0, axis=1)]
    agent_valuesBB1 = agent_valuesBB1[~np.all(agent_valuesBB1 == 0, axis=1)]
    cumm_regretBB1 = cumm_regretBB1[~np.all(cumm_regretBB1 == 0, axis=1)]
    contexts_seenBB1 = contexts_seenBB1[~np.all(contexts_seenBB1 == 0, axis=1)]
    test_expert_valueBB1 = test_expert_valueBB1[~np.all(test_expert_valueBB1 == 0, axis=1)]
    test_agent_valueBB1 = test_agent_valueBB1[~np.all(test_agent_valueBB1 == 0, axis=1)]
    
    expert_values_stdBB1 =     expert_valuesBB1.std(axis=0)
    agent_values_stdBB1 =      agent_valuesBB1.std(axis=0)
    expert_values_meanBB1 =    expert_valuesBB1.mean(axis=0)
    agent_values_meanBB1 =     agent_valuesBB1.mean(axis=0)
    cumm_regret_meanBB1 =      cumm_regretBB1.mean(axis=0)
    cumm_regret_stdBB1 =       cumm_regretBB1.std(axis=0)
    contexts_seen_stdBB1 =     contexts_seenBB1.std(axis=0)
    contexts_seen_meanBB1 =    contexts_seenBB1.mean(axis=0)    

    if TESTS:
        # Value graph (contexts)
        fig = plt.figure(figsize=(4,3))
        p = fig.add_subplot(111)
        
        test_expert_value_meanE = test_expert_valueE.mean(axis=0)
        test_expert_value_stdE = test_expert_valueE.std(axis=0)
        test_agent_value_meanE = test_agent_valueE.mean(axis=0)
        test_agent_value_stdE = test_agent_valueE.std(axis=0)

        p.plot(list(range(0,contexts_seen_minE,sample_step)), test_expert_value_meanE, label="Expert", color='black')
        p.plot(list(range(0,contexts_seen_minE,sample_step)), test_agent_value_meanE, label="Ellipsoid", color='g')

        p.fill_between(list(range(0,contexts_seen_minE,sample_step)), test_expert_value_meanE - test_expert_value_stdE, test_expert_value_meanE + test_expert_value_stdE, color='black', alpha=0.1)
        p.fill_between(list(range(0,contexts_seen_minE,sample_step)), test_agent_value_meanE - test_agent_value_stdE, test_agent_value_meanE + test_agent_value_stdE, color='g', alpha=0.1)

        test_expert_value_meanBB1 = test_expert_valueBB1.mean(axis=0)
        test_expert_value_stdBB1 = test_expert_valueBB1.std(axis=0)
        test_agent_value_meanBB1 = test_agent_valueBB1.mean(axis=0)
        test_agent_value_stdBB1 = test_agent_valueBB1.std(axis=0)

        p.plot(list(range(0,contexts_seen_minBB1,sample_step)), test_agent_value_meanBB1, label="ES", color='r')
        p.fill_between(list(range(0,contexts_seen_minBB1,sample_step)), test_agent_value_meanBB1 - test_agent_value_stdBB1, test_agent_value_meanBB1 + test_agent_value_stdBB1, color='r', alpha=0.1)    
    
        p.set_xlabel("Demonstrations")
        p.set_ylabel("Value")
        p.tick_params('y', colors='k')
        p.set_ylim(0.014,0.021)
#         plt.title("Avg. value on test set")
        p.legend(loc=8)
        plt.show()
        
        minlen_accurE = accurE[0].size
        for i in range(len(accurE)):
            if accurE[i].size < minlen_accurE:
                minlen_accurE = accurE[i].size
        
        accuracyE = np.zeros([len(accurE),minlen_accurE])
        for i in range(len(accurE)):
            accuracyE[i,:] = accurE[i][:minlen_accurE]
    
        minlen_accurBB1 = accurBB1[0].size
        for i in range(len(accurBB1)):
            if accurBB1[i].size < minlen_accurBB1:
                minlen_accurBB1 = accurBB1[i].size
                
        accuracyBB1 = np.zeros([len(accurBB1),minlen_accurBB1])
        for i in range(len(accurBB1)):
            accuracyBB1[i,:] = accurBB1[i][:minlen_accurBB1]
            
        accuracy_meanE = 1 - accuracyE.mean(axis=0)
        accuracy_stdE = accuracyE.std(axis=0)
        accuracy_meanBB1 = 1 - accuracyBB1.mean(axis=0)
        accuracy_stdBB1 = accuracyBB1.std(axis=0)

        fig5 = plt.figure(figsize=(4,3))
        p5 = fig5.add_subplot(111)
        p5.set_xlim(8,200)
        p = p5.twinx()
        lin0 = p.plot(list(range(0,contexts_seen_minE,sample_step)), test_expert_value_meanE, label="Expert", color='black')
        p5.fill_between(list(range(0,contexts_seen_minE,sample_step)), test_expert_value_meanE - test_expert_value_stdE, test_expert_value_meanE + test_expert_value_stdE, color='black', alpha=0.1)
        lin1 = p5.plot(list(range(0,minlen_accurE*sample_step,sample_step)), 100*accuracy_meanE, label="Ellipsoid", color='#145a32')
        p5.fill_between(list(range(0,minlen_accurE*sample_step,sample_step)), 100*accuracy_meanE - 100*accuracy_stdE, 100*accuracy_meanE + 100*accuracy_stdE, color='#145a32', alpha=0.1)
        
        lin2 = p5.plot(list(range(0,minlen_accurBB1*sample_step,sample_step)), 100*accuracy_meanBB1, label="ES", color='#a93226')
        p5.fill_between(list(range(0,minlen_accurBB1*sample_step,sample_step)), 100*accuracy_meanBB1 - 100*accuracy_stdBB1, 100*accuracy_meanBB1 + 100*accuracy_stdBB1, color='#a93226', alpha=0.1)
        
        lin4 = p.plot(list(range(0,contexts_seen_minE,sample_step)), test_agent_value_meanE, label="Ellipsoid", color='g')
        p.fill_between(list(range(0,contexts_seen_minE,sample_step)), test_agent_value_meanE - test_agent_value_stdE, test_agent_value_meanE + test_agent_value_stdE, color='g', alpha=0.1)
        
        lin5 = p.plot(list(range(0,contexts_seen_minBB1,sample_step)), test_agent_value_meanBB1, label="ES", color='r')
        p.fill_between(list(range(0,contexts_seen_minBB1,sample_step)), test_agent_value_meanBB1 - test_agent_value_stdBB1, test_agent_value_meanBB1 + test_agent_value_stdBB1, color='r', alpha=0.1)

        p.set_xlabel("Demonstrations")
        p.set_ylabel("Value")
        p.set_ylim(0.012,0.0208)
#         plt.title("Avg. value on test set")
        linss = lin0+lin4+lin5+lin6+lin1+lin2+lin3
        labss =[linn.get_label() for linn in linss]
        p.legend(linss,labss,loc='center right')
        
        
        p5.set_xlabel("Demonstrations")
        p5.set_ylabel("% Miss")
        p5.set_ylim(10,100)
        p5.tick_params('y', colors='#145a32')
        p.tick_params('y', colors='g')
#         plt.title("Accuracy of learned policy(contexts seen)")
#         p5.legend(loc='center right')
        plt.show()
    
    fig = plt.figure(figsize=(4,3))
    p = fig.add_subplot(111)
    
    p.plot(list(range(iters1)), cumm_regret_meanE, label="Ellipsoid", color='g')
    p.fill_between(list(range(iters1)), cumm_regret_meanE - cumm_regret_stdE, cumm_regret_meanE + cumm_regret_stdE, color='g', alpha=0.1)
    p.plot(list(range(iters2)), cumm_regret_meanBB1, label="ES, loss (1)", color='r')
    p.fill_between(list(range(iters2)), cumm_regret_meanBB1 - cumm_regret_stdBB1, cumm_regret_meanBB1 + cumm_regret_stdBB1, color='r', alpha=0.1)
    
    p.set_xlabel("Time")
    p.set_ylabel("Regret")
    p.legend(loc='best')
#     plt.title("Cumulative regret over time")
    plt.show()


    fig = plt.figure(figsize=(4,3))
    p2 = fig.add_subplot(111)

    p2.plot(list(range(iters1)), contexts_seen_meanE, label="Ellipsoid", color='g')
    p2.fill_between(list(range(iters1)), contexts_seen_meanE - contexts_seen_stdE, contexts_seen_meanE + contexts_seen_stdE, color='g', alpha=0.1)

    
    p2.plot(list(range(iters2)), contexts_seen_meanBB1, label="ES, loss (1)", color='r')
    p2.fill_between(list(range(iters2)), contexts_seen_meanBB1 - contexts_seen_stdBB1, contexts_seen_meanBB1 + contexts_seen_stdBB1, color='r', alpha=0.1)

    p2.set_ylabel("Demonstrations")

    p2.set_xlabel("Time")

#     plt.title("Demonstrations required over time")
    p2.legend(loc='best')

    plt.show()

## Autonomous Driving Simulation

### graphs for the linear case:

In [ ]:
x=make_graphs_drv()

### graphs for the non-linear case:

In [ ]:
y=make_graphs_drv_non_linear()

## Dynamic Treatment Regime

In [ ]:
z=make_graphs_med()